In [0]:
%sql
USE CATALOG vstone_project;
USE SCHEMA db_project;

In [0]:
%sql
-- 1. Create the RLS Filter Function
CREATE OR REPLACE FUNCTION db_project.region_filter(city STRING)
RETURN 
  IS_ACCOUNT_GROUP_MEMBER('admin_group') OR 
  (IS_ACCOUNT_GROUP_MEMBER('analyst_group') AND city IN ('Tashkent', 'Samarkand'));

In [0]:
%sql
-- 2. Create the CLS Masking Function
CREATE OR REPLACE FUNCTION db_project.cost_mask(cost DOUBLE)
RETURN CASE
  WHEN IS_ACCOUNT_GROUP_MEMBER('admin_group') THEN cost
  ELSE NULL 
END;

In [0]:
%sql
-- 3. CREATE THE VIEW (Must happen before GRANTS)
CREATE OR REPLACE VIEW vstone_project.db_project.secure_gold_sales 
AS
SELECT
  city_name,
  db_project.cost_mask(avg_car_price) AS avg_car_price,
  total_listings,
  total_market_value
FROM vstone_project.db_project.gold_sales_by_geography
WHERE db_project.region_filter(city_name);

In [0]:
%sql
-- 4. Apply the GRANTS now that the view exists
GRANT SELECT ON SCHEMA vstone_project.db_project TO `admin_group`;
GRANT SELECT ON SCHEMA vstone_project.db_project TO `analyst_group`;
GRANT SELECT ON VIEW vstone_project.db_project.secure_gold_sales TO `analyst_group`;

In [0]:
%sql
-- Check if the view exists and see the applied comments
DESCRIBE TABLE vstone_project.db_project.secure_gold_sales;

-- Check the masking and filtering functions
SHOW FUNCTIONS IN vstone_project.db_project;

In [0]:
%sql
-- Step 1: Create a physical Shadow Table for Time Travel demonstration
-- This table is separate from your DLT Materialized Views
CREATE OR REPLACE TABLE vstone_project.db_project.gold_sales_audit_history
AS SELECT * FROM vstone_project.db_project.gold_sales_by_geography;


In [0]:
%sql
-- Step 2: View the Version History (This is your key proof for the project)
DESCRIBE HISTORY vstone_project.db_project.gold_sales_audit_history;

In [0]:
%sql
-- Step 3: Demonstrate a Version Query
-- This shows you can query the exact state of the table at Version 0
SELECT * FROM vstone_project.db_project.gold_sales_audit_history VERSION AS OF 0 limit 10;